In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [ ]:
dir=r"/kaggle/input/plant-village"
dataset=tf.keras.preprocessing.image_dataset_from_directory(dir,shuffle=True,batch_size=32)
#batchsize taklen as 32

In [ ]:
class_names=dataset.class_names

data=dataset

In [ ]:
for image_batch,label_batch in data.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

In [ ]:
#if you want to print the first image
for image_batch,label_batch in data.take(1):
    for i in range (5) :
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.xlabel(class_names[label_batch.numpy()[i]])
        plt.show()


In [ ]:
#imp do this for an rgb image
for image_batch,label_batch in data.take(1):
    plt.imshow(image_batch[4].numpy().astype("uint8"))#take this for an rgb image
    plt.xlabel(class_names[label_batch.numpy()[4]])

In [ ]:
train=data.take(int(.8*len(data)))
valid=data.skip(int(.8*len(data)))
test=valid.skip(int(.1*len(data)))
valid=valid.take(int(.1*len(data)))
print(len(train))
print(len(test))
print(len(valid))

In [ ]:
# cache will keep the image in memory and increase performanc
#this whole code is to increase the performance
train=train.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
test=test.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
valid=valid.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)

In [ ]:
resize_rescale=tf.keras.Sequential([
    layers.Rescaling(1/255),
    layers.Resizing(256,256),
])
from tensorflow.keras import layers

# Augmentation pipeline for the emotion detector dataset
augment = tf.keras.Sequential([
  
    layers.RandomFlip(mode='horizontal'),  # Randomly flip inputs horizontally
    layers.RandomRotation(0.2),  # Randomly rotate inputs by up to 20 degrees
    layers.RandomZoom(height_factor=0.2, width_factor=0.2)  # Randomly zoom images
])


In [ ]:
model=tf.keras.models.Sequential ([
    resize_rescale,
    augment,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(32,256,2,3)),#batch size 32
    layers.MaxPool2D(2),
     layers.Conv2D(32,(3,3),activation='relu'),#32 came from trial and error
    layers.MaxPool2D(2),
     layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPool2D(2),
     layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPool2D(2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(100,activation='relu'),
    layers.Dense(len(class_names),activation='softmax')
])

In [ ]:
model.build(input_shape=(32,256,256,3))

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [ ]:
history=model.fit(train,epochs=20,batch_size=32,validation_data=valid)

In [ ]:
model.evaluate(test)

In [ ]:
for batch,label in test.take(1) :
    print(class_names[label[5]])
    pred=model.predict(batch)
    print(class_names[np.argmax(pred[5])])
    plt.imshow(batch[5].numpy().astype("uint8"))

In [ ]:
for i,l in test.take(1):
    print(class_names[l[0]])
    print()

In [ ]:
def predict(model,im):
    pred=class_names[np.argmax(model.predict(tf.expand_dims(im,axis=0)))]
    confidence=round(np.max(model.predict(tf.expand_dims(im,axis=0)))*100,2)
    return pred,confidence

In [ ]:

plt.figure(figsize=(15, 15))
for images, labels in test.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class,c=predict(model,images[i])
        actual_class = class_names[labels[i]] 
       
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {c}%")
        
        plt.axis("off")
        

In [ ]:
d=r"/kaggle/working/ayush.keras"
model.save(d)
